In [14]:
import os

# Define the input file path
#Input_file_path = "D:/Halima's Data/Thesis_2/RCode/1_GD428_21136_Hu_REH_Parental.h5ad"


Input_file_path = "D:/Halima's Data/Thesis_2/RCode/2_GD444_21136_Hu_Sup_Parental.h5ad"

#Input_file_path = "D:/Halima's Data/Thesis_2/RCode/8_10-k-human-pbm-cs-multiome-v-1-0-chromium-controller.h5ad"

# Split the path to separate the directory and the filename
directory, filename = os.path.split(Input_file_path)



print(filename)  # This will print 'RCode'


2_GD444_21136_Hu_Sup_Parental.h5ad


In [15]:
def confusionMatrix(a1, b1):
    v1 = set(a1)
    v2 = set(b1)
    dfTmp = pd.DataFrame([pd.Series(list(a1)),pd.Series(list(b1))]).T
    df1 = pd.DataFrame(index=v1, columns = v2)
    for i in v1:
        for j in v2:
            df1.loc[i,j] = dfTmp.loc[(dfTmp[0]==i) & (dfTmp[1]==j)].shape[0]
    return df1

In [1]:
from os.path import exists
import numpy as np
import pandas as pd
#import pickle

# Single Cell Packages
import scanpy as sc

# Import ccAF for classification of cell cycle phase
import ccAF

In [17]:
## 2. Load up conversion dictionaries for gene symbol to Ensembl gene ID
# HGNC -> downlaoded from HGNC website (https://www.genenames.org/download/custom/)
hgncEnsmbl = pd.read_csv("D:/Halima's Data/Thesis_2/RCode/data/geneConversions/hgnc_geneSymbols_ensmbl.txt", index_col=1, header=0, sep='\t')
hgncEnsmbl = hgncEnsmbl.loc[~hgncEnsmbl['Ensembl ID(supplied by Ensembl)'].isnull()]

In [18]:
ensmblHgnc = pd.Series(hgncEnsmbl.index)
ensmblHgnc.index = list(hgncEnsmbl['Ensembl ID(supplied by Ensembl)'])

In [19]:

hgncPrevEnsmbl = {}
for i in hgncEnsmbl.loc[~hgncEnsmbl['Previous symbols'].isnull()].index:
    splitUp = hgncEnsmbl.loc[i,'Previous symbols'].split(', ')
    ensmbl = hgncEnsmbl.loc[i,'Ensembl ID(supplied by Ensembl)']
    for j in splitUp:
        hgncPrevEnsmbl[j] = ensmbl

hgncAliasEnsmbl = {}
for i in hgncEnsmbl.loc[~hgncEnsmbl['Alias symbols'].isnull()].index:
    splitUp = hgncEnsmbl.loc[i,'Alias symbols'].split(', ')
    ensmbl = hgncEnsmbl.loc[i,'Ensembl ID(supplied by Ensembl)']
    for j in splitUp:
        hgncAliasEnsmbl[j] = ensmbl

In [20]:
'''
from itertools import islice


# Print first 10 items
for key, value in islice(hgncPrevEnsmbl.items(), 10):
    print(key, value)  '''

'\nfrom itertools import islice\n\n\n# Print first 10 items\nfor key, value in islice(hgncPrevEnsmbl.items(), 10):\n    print(key, value)  '

In [21]:
'''

# Print first 10 items
for key, value in islice(hgncAliasEnsmbl.items(), 10):
    print(key, value)
    
    '''

'\n\n# Print first 10 items\nfor key, value in islice(hgncAliasEnsmbl.items(), 10):\n    print(key, value)\n    \n    '

In [22]:


adata1 = sc.read_h5ad(Input_file_path)

print(adata1.obs_names[:5])  # First 5 cell IDs
print(adata1.var_names[:5])  # First 5 gene names
print(adata1)

Index(['AAACAGCCACATTGCA-1', 'AAACAGCCAGATAGAC-1', 'AAACATGCAAGGTAAC-1',
       'AAACATGCATTAAGTC-1', 'AAACCAACACAGGGAC-1'],
      dtype='object')
Index(['AL627309.1', 'AL627309.5', 'AP006222.2', 'AL669831.2', 'LINC01409'], dtype='object')
AnnData object with n_obs × n_vars = 5239 × 23288
    obs: 'ClusterID', 'ClusterName', 'clusts_WT', 'nCount_RNA', 'nFeature_RNA', 'nGene', 'nUMI', 'orig_ident', 'percent_mito', 'pert'
    var: 'Selected'
    obsm: 'pca_cell_embeddings', 'umap_cell_embeddings'
    layers: 'norm_data', 'scale_data'


In [23]:
## 4. Convert to Ensembl gene IDs
# Convert from gene symbols to Ensmbl gene IDs
adata1 = adata1[:,[True if i in hgncEnsmbl.index or i in hgncPrevEnsmbl or i in hgncAliasEnsmbl else False for i in adata1.var_names]]
convertMe_hgnc = pd.DataFrame(np.nan, index=adata1.var_names, columns=['ensembl'])
numConverted_hgnc = 0
missed = []
for j in adata1.var_names:
    if j in hgncEnsmbl.index:
        convertMe_hgnc.loc[j,'ensembl'] = hgncEnsmbl.loc[j,'Ensembl ID(supplied by Ensembl)']
    elif j in hgncPrevEnsmbl:
        convertMe_hgnc.loc[j,'ensembl'] = hgncPrevEnsmbl[j]
    elif j in hgncAliasEnsmbl:
        convertMe_hgnc.loc[j,'ensembl'] = hgncAliasEnsmbl[j]
    else:
        missed.append(j)

adata1.var_names = pd.Index(convertMe_hgnc['ensembl'])

In [24]:
print("Check again!")
print(adata1)
print(adata1.obs_names[:5])  # First 5 cell IDs
print(adata1.var_names[:5])  # First 5 gene names

Check again!
AnnData object with n_obs × n_vars = 5239 × 17370
    obs: 'ClusterID', 'ClusterName', 'clusts_WT', 'nCount_RNA', 'nFeature_RNA', 'nGene', 'nUMI', 'orig_ident', 'percent_mito', 'pert'
    var: 'Selected'
    obsm: 'pca_cell_embeddings', 'umap_cell_embeddings'
    layers: 'norm_data', 'scale_data'
Index(['AAACAGCCACATTGCA-1', 'AAACAGCCAGATAGAC-1', 'AAACATGCAAGGTAAC-1',
       'AAACATGCATTAAGTC-1', 'AAACCAACACAGGGAC-1'],
      dtype='object')
Index(['ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794',
       'ENSG00000225880', 'ENSG00000230368'],
      dtype='object', name='ensembl')


In [25]:
## 5. ACTINN Classification
# Run ccAF ACTINN classifier against data
print('\nACTINN...')
predictedLabels = ccAF.ccAF.predict_labels(adata1)
adata1.obs['ccAF'] = predictedLabels

new_edited_anndata = "./WT_"+filename
adata1.write(filename=new_edited_anndata)


ACTINN...
Sparse False  Index(['AAACAGCCACATTGCA-1', 'AAACAGCCAGATAGAC-1', 'AAACATGCAAGGTAAC-1',
       'AAACATGCATTAAGTC-1', 'AAACCAACACAGGGAC-1', 'AAACCGAAGAGAAGGG-1',
       'AAACCGAAGGATCCGC-1', 'AAACCGAAGTTAGCCG-1', 'AAACCGCGTGTGAGAG-1',
       'AAACCGCGTTAATCGG-1',
       ...
       'TTTGTGGCAATTGAAG-1', 'TTTGTGGCACAGACTC-1', 'TTTGTGGCACTTGTTC-1',
       'TTTGTGGCATATTGAC-1', 'TTTGTGGCATCACAGC-1', 'TTTGTGTTCACTAAGC-1',
       'TTTGTTGGTAACGTGC-1', 'TTTGTTGGTTCGCTCA-1', 'TTTGTTGGTTGCTGGG-1',
       'TTTGTTGGTTTGGCGG-1'],
      dtype='object', length=5239) Index(['ENSG00000237491', 'ENSG00000177757', 'ENSG00000228794',
       'ENSG00000225880', 'ENSG00000230368', 'ENSG00000223764',
       'ENSG00000187634', 'ENSG00000188976', 'ENSG00000187961',
       'ENSG00000187583',
       ...
       'ENSG00000198712', 'ENSG00000228253', 'ENSG00000198899',
       'ENSG00000198938', 'ENSG00000198840', 'ENSG00000212907',
       'ENSG00000198886', 'ENSG00000198786', 'ENSG00000198695',
       'ENS

In [26]:
## 6. Downstream analyses
# Dataframe of true labels, predictions, probabilities for all iterations
DF = pd.concat([adata1.obs, pd.DataFrame({'Predictions':predictedLabels},index=adata1.obs_names) ], axis=1)

new_csv_file = "./ccaf_result_"+filename+".csv"
DF.to_csv(new_csv_file)